In [33]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import numbers
import decimal
import scipy.stats as ss
import matplotlib.pyplot as plt
from statistics import stdev
from statistics import mean
import time
import math
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
stop_words=set(stopwords.words("english"))
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from models.naive_bayes import MultiNomialNB as CustomNaiveBayes
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
def getScoretWithModel(model, x_train, x_test, y_train, y_test):
    model.fit(x_train, y_train)
    return model.score(x_test, y_test)

In [12]:
redditDataTrain = pd.read_csv("data/reddit_train.csv") #, sep="\n", header=None) 
redditDataTest = pd.read_csv("data/reddit_test.csv") # sep="\n", header=None)
redditDataTrain

,id,comments,subreddits
0,0,"Honestly, Buffalo is the correct answer. I rem...",hockey
1,1,Ah yes way could have been :( remember when he...,nba
2,2,https://youtu.be/6xxbBR8iSZ0?t=40m49s\n\nIf yo...,leagueoflegends
3,3,He wouldn't have been a bad signing if we woul...,soccer
4,4,Easy. You use the piss and dry technique. Let ...,funny
...,...,...,...
69995,69995,"Thank you, you confirm Spain does have nice pe...",europe
69996,69996,Imagine how many he would have killed with a r...,leagueoflegends
69997,69997,Yes. Only. As in the guy I was replying to was...,canada
69998,69998,Looking for something light-hearted or has a v...,anime


In [13]:
commentsTrain = redditDataTrain.iloc[:,1]
subredditsTrain = redditDataTrain.iloc[:,-1]
commentsTest = redditDataTest.iloc[:,1]

In [15]:
tfidf = TfidfVectorizer(stop_words='english', )
cv = CountVectorizer()
lr = LogisticRegression()
multiNB = MultinomialNB()
dtc = tree.DecisionTreeClassifier()
kf = StratifiedKFold(n_splits=5)
cnb = CustomNaiveBayes()

smoothing factor 0.01


In [16]:
GX_train, GX_test, Gy_train, Gy_test = train_test_split(commentsTrain, subredditsTrain, test_size=0.2)

In [17]:
# GX_train = commentsTrain[commentsTrain.index < np.percentile(commentsTrain.index, 80)].sort_index()
# Gy_train = subredditsTrain[subredditsTrain.index < np.percentile(subredditsTrain.index, 80)].sort_index()
# GX_test = commentsTrain[commentsTrain.index > np.percentile(commentsTrain.index, 80)].sort_index()
# Gy_test = subredditsTrain[subredditsTrain.index > np.percentile(subredditsTrain.index, 80)].sort_index()

### Experimentation

In [18]:
kf = StratifiedKFold(n_splits=4)
def get_avg_acc_for_model(model, **kwargs):
    results = []
    for train_index, test_index in kf.split(GX_train, Gy_train):
        x_train, x_test, y_train, y_test = GX_train[train_index], GX_train[test_index], Gy_train[train_index], Gy_train[test_index]
        redditDataTrainTF = tfidf.fit_transform(x_train)
        redditDataTestTF = tfidf.transform(x_test)
        redditDataTrainTF.toarray()
        clf = model(**kwargs)
        results.append(getScoretWithModel(clf, redditDataTrainTF, redditDataTestTF, y_train, y_test))
    avg_acc = sum(results)/len(results)
    return avg_acc

In [25]:
tfidf = TfidfVectorizer(stop_words='english', strip_accents='ascii', ngram_range=(1, 2))
get_avg_acc_for_model(LogisticRegression)

/Users/faizank/Documents/courses/comp551/mini_project1/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/faizank/Documents/courses/comp551/mini_project1/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/faizank/Documents/courses/comp551/mini_project1/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/faizank/Documents/courses/comp551/mini_project1/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to si

0.5239464285714286

In [14]:
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
tfidf = TfidfVectorizer(stop_words=stop_words, strip_accents='ascii', ngram_range=(1, 1), lowercase=True, tokenizer = token.tokenize)

In [37]:
GX_train_idf = tfidf.fit_transform(GX_train)
GX_test_idf = tfidf.transform(GX_test)
clf = LogisticRegression()
final_acc = getScoretWithModel(clf, GX_train_idf, GX_test_idf, Gy_train, Gy_test)
print(final_acc)

/Users/faizank/Documents/courses/comp551/mini_project1/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/faizank/Documents/courses/comp551/mini_project1/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.5447142857142857


In [18]:
GX_train_idf = tfidf.fit_transform(GX_train)
GX_test_idf = tfidf.transform(GX_test)
clf = MultinomialNB()
final_acc = getScoretWithModel(clf, GX_train_idf, GX_test_idf, Gy_train, Gy_test)
print(final_acc)

/Users/faizank/Documents/courses/comp551/mini_project1/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/faizank/Documents/courses/comp551/mini_project1/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.5370714285714285


In [12]:
GX_train_idf = tfidf.fit_transform(GX_train)
GX_test_idf = tfidf.transform(GX_test)
clf = RandomForestClassifier()
final_acc = getScoretWithModel(clf, GX_train_idf, GX_test_idf, Gy_train, Gy_test)
print(final_acc)

/Users/faizank/Documents/courses/comp551/mini_project1/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.40264285714285714


In [11]:
GX_train_idf = tfidf.fit_transform(GX_train)
GX_test_idf = tfidf.transform(GX_test)
clf = LinearSVC()
final_acc = getScoretWithModel(clf, GX_train_idf, GX_test_idf, Gy_train, Gy_test)
print(final_acc)

0.5430714285714285


##### best model so far

In [ ]:
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
tfidf = TfidfVectorizer(stop_words=stop_words, strip_accents='ascii', ngram_range=(1, 1), lowercase=True, tokenizer = token.tokenize)
GX_train_idf = tfidf.fit_transform(GX_train)
GX_test_idf = tfidf.transform(GX_test)
clf = CustomNaiveBayes(smoothing_factor=0.1)
final_acc = getScoretWithModel(clf, GX_train_idf, GX_test_idf, Gy_train, Gy_test)
print(final_acc)

In [18]:
tfidf = TfidfVectorizer(stop_words=stop_words, strip_accents='ascii', ngram_range=(1, 1), lowercase=True, tokenizer = token.tokenize)
GX_train_idf = tfidf.fit_transform(GX_train)
GX_train_idf.shape

(56000, 64893)

In [23]:
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
tfidf = TfidfVectorizer(stop_words=stop_words, strip_accents='ascii', ngram_range=(1, 2), lowercase=True, tokenizer = token.tokenize)
GX_train_idf = tfidf.fit_transform(GX_train)
GX_test_idf = tfidf.transform(GX_test)
GX_train_idf.shape

(56000, 901866)

In [31]:
chi2_selector = SelectKBest(chi2, k=700000)
GX_train_idf_best = chi2_selector.fit_transform(GX_train_idf, Gy_train)
GX_test_idf_best = chi2_selector.fit_transform(GX_test_idf, Gy_test)
GX_test_idf_best.shape

(14000, 700000)

In [34]:
clf = RandomForestClassifier()
final_acc = getScoretWithModel(clf, GX_train_idf_best, GX_test_idf_best, Gy_train, Gy_test)
print(final_acc)

/Users/faizank/Documents/courses/comp551/mini_project1/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.05228571428571428
